In [1]:
import pandas as pd
import pandas_profiling

data = pd.read_csv('./reviews.csv')

In [3]:
data.head(1)

,star,date,vendor,review,help
0,5,2022.03.12,쿠팡(주),굿굿 ♡ ♥ 내돈내산 솔직담백 리뷰 입니다 ! ♥ 항상 제품을 구매하기전 후기를...,12


In [4]:
len(data) # 총 192977개의 리뷰를 수집했다.

192977

In [6]:
import re
from hanspell import spell_checker

In [7]:
def find_text(text):
    pattern = '[^\w\s]'         # 특수기호제거'[^\w\sㄱ-ㅎㅏ-ㅣ]' 
    text = re.sub(pattern=pattern, repl='', string=text)
    text = re.sub(pattern='[\s]+', repl=' ', string=text)

    return text 

In [8]:
from hanspell import spell_checker
def data_preprocessing(review):
    return spell_checker.check(review).checked

In [ ]:
data['review'] = data['review'].apply(find_text)
data['review_r'] = ''

In [ ]:
for idx in range(105867, 110001): 
    # 이런 식으로 10만 단위씩 끊어서 실행 및 저장. 
    # 에러 시 해당 번호(idx)부터 재실행
    # 하다가 빈번한 에러로 인해 try-except 적용
    # 미리 공백('')을 기본값으로 생성했기 때문에 큰 문제 없음
    try :
        data['review_r'][idx] = spell_checker.check(data.loc[idx,'review']).checked
    except : pass
    print(idx)
data.to_csv('./reviews_r.csv')    

In [ ]:
import numpy as np
for idx, d in enumerate(data['review_r']):
    if len(d) <1 : 
        print(d)
        data['review_r'][idx] = np.nan
# 공백 제거가 안 돼서 다시 Null로 만들었다.

In [ ]:
data.info()
'''
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146558 entries, 0 to 146557
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   star      146558 non-null  int64 
 1   date      146558 non-null  object
 2   vendor    146558 non-null  object
 3   review    146558 non-null  object
 4   help      146558 non-null  int64 
 5   review_r  144882 non-null  object
dtypes: int64(2), object(4)
'''

In [ ]:
data.dropna(subset='review_r', inplace=True)
data.reset_index(inplace=True)
del data['index']

In [ ]:
# 데이터 전처리된 컬럼 review_r을 만들었다.

In [ ]:
from konlpy.tag import Okt

okt = Okt()

In [ ]:
data['tokenized'] = data['review_r'].apply(lambda x: okt.morphs(x))

In [ ]:
data['tokenized'][0] # 형태소 단위로 잘 끊겨서 리스트로 담겨 있다.

In [ ]:
from collections import Counter

In [ ]:
noun_list = np.hstack(data['tokenized'].values).tolist()
result = ' '.join(s for s in noun_list)
# 단어 빈도를 확인하기 위해 하나의 문자열로 만들었다.
count = Counter(okt.nouns(result))
noun_list_1000 = count.most_common(1000)
for w in noun_list_1000:
    print(w)
# 근데 봐도 잘 모르겠음

In [ ]:
data.to_csv('./reviews_token.csv', index=False) 

새 파일로 작업했다.

In [ ]:
import pandas as pd
import re

In [ ]:
data = pd.read_csv('./reviews_token.csv')
data.head()

In [ ]:
len(data)

In [ ]:
word_list = pd.read_csv('./word_dict.csv')
word_list
# 엑셀로 만든 단어 사전 리스트를 읽어 왔다.
# [word, pos, neg, rat, sign]
# 단어, 긍정가중치합, 부정가중치합, 긍정/부정 또는 부정/긍정 비율, 긍정이 많을 경우 +, 부정이 많을 경우 -의 정보로 구성되어 있다.

In [ ]:
'''
참고로 pos와 neg는 이렇게 구했다.
1) 일단 구하기
negative_point = 0.0
positive_point = 0.0

for idx, word in enumerate(word_list) :
    try:
        for i in range(len(data)): 
            if re.search(word, data['review_r'].iloc[i]) or re.search(word, data['review'].iloc[i]):
                if data.iloc[i]['star'] == 5 : 
                    positive_point += 4/101203*144882
                elif data.iloc[i]['star'] == 4 :
                    positive_point += 3/21465*144882
                    negative_point += 1/21465*144882 
                elif data.iloc[i]['star'] == 3 :
                    positive_point += 2/8661*144882 
                    negative_point += 2/8661*144882 
                elif data.iloc[i]['star'] == 2 :
                    positive_point += 1/3809*144882 
                    negative_point += 3/3809*144882 
                elif data.iloc[i]['star'] == 1 :
                    negative_point += 4/9744*144882   
                    
        dictionary_df.iloc[idx,1] = positive_point
        dictionary_df.iloc[idx,2] = negative_point
        negative_point = 0.0
        positive_point = 0.0
    except Exception as e : print(idx, e)
    
2) 조건 미달 제거하기
dictionary_df['rat'] = 0.0
idx_list = []
for idx in range(len(dictionary_df)):
    if dictionary_df.iloc[idx,2]>dictionary_df.iloc[idx,1]:
        dictionary_df.iloc[idx,3] = dictionary_df.iloc[idx,2]/dictionary_df.iloc[idx,1]
    else :
        dictionary_df.iloc[idx,3] = dictionary_df.iloc[idx,1]/dictionary_df.iloc[idx,2]
    if dictionary_df.iloc[idx,3] < 2 or dictionary_df.iloc[idx,3] == float('inf') or (dictionary_df.iloc[idx,2]<1000 and dictionary_df.iloc[idx,1]):
        idx_list.append(idx)

save = dictionary_df.drop(idx_list)

3) 부호 구하기
save['sign'] = 1
for idx in range(len(save)):
    save['sign'].iloc[idx] = 1 if save['pos'].iloc[idx] >= save['neg'].iloc[idx] else -1

save['sign'].value_counts()
save.to_csv('./point_dict_2.0.csv', index=False)

4) 유효성 확인하기
data['valid'] = 0
data['star'].value_counts()

for i in range(len(data)): # 4나 5는 긍정, 1, 2, 3은 부정으로 임의 처리
    if data['label'].iloc[i] == 1 and (data['star'].iloc[i] == 4 or data['star'].iloc[i] == 5) :
        data['valid'].iloc[i] = 1
    elif data['label'].iloc[i] == 0 and (data['star'].iloc[i] == 1 or data['star'].iloc[i] == 2 or data['star'].iloc[i] == 3):
        data['valid'].iloc[i] = 1
        
data['valid'].value_counts()
valid_data = data[data['valid']==1]
valid_data['star'].value_counts() # 일치하는 데이터만 쓸까 생각 중
valid_data.to_csv('./valid_data.csv', index=False)
'''

In [ ]:
dictionary_df = pd.DataFrame(word_list['word'].tolist(), columns=['word'])
dictionary_df['pos'] = 0.0
dictionary_df['neg'] = 0.0
dictionary_df
# 야매로 만들었다.

In [ ]:
data['star'].value_counts()
# 별점의 비율을 확인해보았다.

In [ ]:
data['label'] = 1 # 리뷰에서 긍정적인 리뷰가 더 많은 편이어서 기본값을 1로 주었다.

In [ ]:
for i in range(len(data)): 
    point = 0
    for idx, (word, w) in enumerate(zip(word_list['word'], word_list['sign'])) :
        if re.search(word, data['review_r'].iloc[i]) or re.search(word, data['review'].iloc[i]):
            point += w #* rat
    if point >= 0 :
        data['label'].iloc[i] = 1
    else : 
        data['label'].iloc[i] = 0
# 전처리된 리뷰 또는 원본 리뷰에서 해당 단어를 확인해서 확인되는 경우 가중치(- 또는 +)를 누적 합산하도록 했다.

하다가 데이터 라벨링에 대한 고민 끝에 갈아 엎고 5를 긍정, 1, 2를 부정으로 하여 재시작해보도록 했다.

In [ ]:
data = pd.read_csv('./reviews_token.csv')
data['label'] = 0

In [ ]:
# 라벨링
for i in range(len(data)):
    if data['star'].iloc[i] == 5 :
        data['label'].iloc[i] = 1
    elif data['star'].iloc[i] == 1 or data['star'].iloc[i] == 2 : #or data['star'].iloc[i] == 3 or data['star'].iloc[i] == 4
        data['label'].iloc[i] = 0

In [ ]:
# 조건에 맞는 데이터만 사용하기
data2 = data[data['star']==1]
data2 = data2.append(data[data['star']==2])
data2 = data2.append(data[data['star']==5])
data2.reset_index(inplace=True)
data2
# 14만여 개 중 약 3만 건이 제외된다. (3, 4 별점 데이터)
# 총 114756 개 데이터
# 1    101203, 0     13553 (긍정댓글 : 부정댓글)

In [ ]:
from transformers import BertTokenizer

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
from kss import split_sentences

In [ ]:
tokenized = pd.DataFrame(columns=['sents', 'label'])
tokenized

In [ ]:
for sents, label in zip(data2['review_r'], data2['label']):
  for sent in split_sentences(sents):
#  tokenized.append(bert_tokenizer(sent))
    t = bert_tokenizer.encode_plus(sent,
                                  add_special_tokens=True,
                                  max_length=30,
                                  padding='max_length',
                                  return_attention_mask=True)
    tokenized = pd.concat([tokenized, pd.Series([t, label])])
# split_sentences로 문장 단위로 끊어서 한 문장씩 토크나이즈 했다.

In [ ]:
tokenized.iloc[0:5]